# 인공지능 작사가 만들기

In [64]:
# 데이터 다운 받기

## 데이터 읽어오기

In [20]:
import numpy as np         # 변환된 문장 데이터(행렬)을 편하게 처리하기 위해
import tensorflow as tf    # 대망의 텐서플로우!
from sklearn.model_selection import train_test_split

import glob  # glob 모듈을 사용하면 파일을 읽어오는 작업을 하기가 아주 용이해요.
import os
import re  ## 정규표현식을 위한 Regex 지원 모듈 (문장 데이터를 정돈하기 위해) 

In [2]:

txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)    #glob를 활용하여 모든 txt파일 읽어오기

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ['\ufeffbaby It was all a dream', 'I used to read Word Up magazine', 'Salt n Pepa and Heavy D up in the limousine']


### 데이터 정제 
- 정제함수로 만든 데이터셋 -> 토큰화 진행  
  -> end 없애서 입력(소스문장), start없애서 출력정답(타겟문장) 만들기
- preprocess_sentence() 함수 이용 
- 토큰화 했을 때 토큰의 갯수가 15개 넘어가는 문장은 학습데이터에서 제외

In [4]:
import re  ## 정규표현식을 위한 Regex 지원 모듈 (문장 데이터를 정돈하기 위해) 
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()       # 소문자로 바꾸고 양쪽 공백을 삭제
  
    # 아래 3단계를 거쳐 sentence는 스페이스 1개를 delimeter로 하는 소문자 단어 시퀀스로 바뀝니다.
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)        # 패턴의 특수문자를 만나면 특수문자 양쪽에 공백을 추가
    sentence = re.sub(r'[" "]+', " ", sentence)                  # 공백 패턴을 만나면 스페이스 1개로 치환
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence)  # a-zA-Z?.!,¿ 패턴을 제외한 모든 문자(공백문자까지도)를 스페이스 1개로 치환

    sentence = sentence.strip()

    sentence = '<start> ' + sentence + ' <end>'      # 이전 스텝에서 본 것처럼 문장 앞뒤로 <start>와 <end>를 단어처럼 붙여 줍니다
    
    return sentence

print(preprocess_sentence("This @_is ;;;sample        sentence."))   # 이 문장이 어떻게 필터링되는지 확인해 보세요.

<start> this is sample sentence . <end>


In [5]:
# 정제함수 활용하여 토큰화 진행 후 <end> 없애고 소스문장, <start> 없애고 타겟문장 : 정제 데이터 구축

corpus = []

for sentence in raw_corpus:
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue
        
    corpus.append(preprocess_sentence(sentence))
        
corpus[:10]

['<start> baby it was all a dream <end>',
 '<start> i used to read word up magazine <end>',
 '<start> salt n pepa and heavy d up in the limousine <end>',
 '<start> hangin pictures on my wall <end>',
 '<start> every saturday rap attack mr magic marley marl <end>',
 '<start> i let my tape rock til my tape popped <end>',
 '<start> smokin weed and bambu sippin on private stock <end>',
 '<start> way back when i had the red and black lumberjack <end>',
 '<start> with the hat to match <end>',
 '<start> remember rappin duke duhha duhha <end>']

### 토큰화 - 단어 사전 만들기 위한 문장 쪼개기
**tf.keras.preprocessing.text.Tokenizer 패키지**
- 정제된 데이터를 토큰화하고, 단어사전dictionary를 만들어 데이터를 숫자로 변환

In [36]:
import tensorflow as tf
def tokenize(corpus):
    # 텐서플로우에서 제공하는 Tokenizer 패키지를 생성
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=7000,  # 전체 단어의 개수 
        filters=' ',    # 별도로 전처리 로직을 추가할 수 있습니다. 이번에는 사용하지 않겠습니다.
        oov_token="<unk>"  # out-of-vocabulary, 사전에 없었던 단어는 어떤 토큰으로 대체할지
    )
    
    tokenizer.fit_on_texts(corpus)   # 우리가 구축한 corpus로부터 Tokenizer가 사전을 자동구축하게 됩니다.


    # 이후 tokenizer를 활용하여 모델에 입력할 데이터셋을 구축하게 됩니다.
    tensor = tokenizer.texts_to_sequences(corpus)   # tokenizer는 구축한 사전으로부터 corpus를 해석해 Tensor로 변환합니다.

    # 입력 데이터의 시퀀스 길이를 일정하게 맞추기 위한 padding  메소드를 제공합니다.
    # maxlen의 디폴트값은 None입니다. 이 경우 corpus의 가장 긴 문장을 기준으로 시퀀스 길이가 맞춰집니다.
    # 토큰화 했을 때 최대 토큰의 갯수가 15개 
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post', maxlen=15)  
   
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   2   51   11 ...    0    0    0]
 [   2    5  291 ...    0    0    0]
 [   2 3196  489 ...    0    0    0]
 ...
 [   2  559   21 ...    0    0    0]
 [   2  120   34 ...    0    0    0]
 [   2    5   22 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7f1410068f50>


In [37]:
print(tensor[:3, :10])

[[   2   51   11   57   24    9  339    3    0    0]
 [   2    5  291   10  629  475   29 2142    3    0]
 [   2 3196  489    1    8 1150  127   29   14    6]]


In [38]:
# 단어 사전이 어떻게 구축 되어있을까?
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : i
6 : the
7 : you
8 : and
9 : a
10 : to


In [39]:
src_input = tensor[:, :-1]  # tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다. 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높습니다.
tgt_input = tensor[:, 1:]    # tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.

print(src_input[0])
print(tgt_input[0])

[  2  51  11  57  24   9 339   3   0   0   0   0   0   0]
[ 51  11  57  24   9 339   3   0   0   0   0   0   0   0]


## 평가 데이터셋 분리
- 훈련 데이터와 평가 데이터 분리
- tokenize() 함수로 데이터를 Tensor로 변환
- sklearn모듈의 train_test_split()함수를 통해 분리하기
- 단어장의 크기는 12,000이상 , 총데이터의 20%를 평가 데이터셋

In [40]:
enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, tgt_input, test_size = 0.2, random_state= 30)

In [41]:
print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

Source Train: (140599, 14)
Target Train: (140599, 14)


In [42]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

VOCAB_SIZE = tokenizer.num_words + 1    # tokenizer가 구축한 단어사전 7000개, 여기 포함되지 않은 0:<pad>를 포함하여 7000+1개

dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>

## 인공지능 학습시키기
- tf.keras.Model을 Subclassing하는 방식 
- 우리가 만들 모델에는 1개의 Embedding 레이어, 2개의 LSTM 레이어, 1개의 Dense 레이어로 구성
    - 임베딩 레이어 : 인덱스 값을 해당 인덱스 번째의 워드 벡터로 바꿔준다. 워드벡터는 의미 벡터 공간에서 단어의 추상적 표현으로 사용됨   
    
### 모델 만들기

In [43]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super(TextGenerator, self).__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256    #조절하기
hidden_size = 1024      #조절하기
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [44]:
for src_sample, tgt_sample in dataset.take(1): break
model(src_sample)

<tf.Tensor: shape=(256, 14, 7001), dtype=float32, numpy=
array([[[ 1.2109203e-05,  5.6750279e-05, -2.3534894e-04, ...,
         -4.3504017e-05, -7.5439908e-05,  7.9272511e-05],
        [ 3.6484317e-04,  3.8952802e-04, -4.5009708e-04, ...,
          2.4349141e-04, -5.5293948e-04,  3.1112591e-04],
        [ 5.3157052e-04,  1.3991776e-04, -3.8912683e-04, ...,
          7.7537552e-05, -6.1225687e-04,  4.0805570e-04],
        ...,
        [ 4.7662060e-04, -6.9115697e-05, -4.4430653e-04, ...,
          4.9907900e-04, -7.5645593e-04, -7.1980181e-04],
        [ 1.1444630e-03,  4.6565401e-05, -6.0193613e-04, ...,
          6.0829957e-04, -1.2109221e-03, -4.2494474e-04],
        [ 1.7818662e-03,  1.8902571e-04, -7.3691935e-04, ...,
          7.6201378e-04, -1.6532571e-03, -1.3339329e-04]],

       [[ 1.2109203e-05,  5.6750279e-05, -2.3534894e-04, ...,
         -4.3504017e-05, -7.5439908e-05,  7.9272511e-05],
        [-5.3298711e-05,  3.5347781e-04, -6.9861003e-04, ...,
          2.4395630e-04, -

<tf.Tensor: shape=(256, 14, 7001), dtype=float32, numpy=

- 1) 7001 - Dense 레이어의 출력 차원수  :7001개 단어 중 어느 단어의 확률이 가장 높을지 모델링  
- 2) 256 - 이전 스텝에서 지정한 배치 사이즈 : dataset.take(1)을 통해 1개의 배치=256개의 문장 데이터 가져옴  
- 3) 14 - LSTM은 자신에게 입력 된 시퀸스의 길이만큼 동일한 길이의 시퀸스를 출력한다.
    - tf.keras.layers.LSTM(hidden_size, return_sequences=True)로 호출한 LSTM 레이어에서 return_sequences=True이라고 지정한 부분
    - 만약 return_sequences=False였다면 LSTM 레이어는 1개의 벡터만 출력
    - 데이터를 입력받으면서 비로소 알게 된 것/우리 데이터셋의 max_len이 15로 맞춰져 있었던 것
        - 토큰 갯수가 15개가 넘어가면 학습데이터에서 제외

In [45]:
model.summary()

Model: "text_generator_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      multiple                  1792256   
_________________________________________________________________
lstm_4 (LSTM)                multiple                  5246976   
_________________________________________________________________
lstm_5 (LSTM)                multiple                  8392704   
_________________________________________________________________
dense_2 (Dense)              multiple                  7176025   
Total params: 22,607,961
Trainable params: 22,607,961
Non-trainable params: 0
_________________________________________________________________


### 모델 학습시키기 - loss 2.2로 줄이기

In [49]:
## 모델 학습
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, 
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, epochs=10,validation_data=dataset)

Epoch 1/10
686/686 [==============================] - 112s 163ms/step - loss: 2.0778 - val_loss: 1.9516
Epoch 2/10
686/686 [==============================] - 115s 168ms/step - loss: 1.9692 - val_loss: 1.8643
Epoch 3/10
686/686 [==============================] - 113s 165ms/step - loss: 1.8879 - val_loss: 1.7817
Epoch 4/10
686/686 [==============================] - 113s 164ms/step - loss: 1.8095 - val_loss: 1.7070
Epoch 5/10
686/686 [==============================] - 113s 165ms/step - loss: 1.7328 - val_loss: 1.6320
Epoch 6/10
686/686 [==============================] - 114s 165ms/step - loss: 1.6599 - val_loss: 1.5564
Epoch 7/10
686/686 [==============================] - 113s 165ms/step - loss: 1.5898 - val_loss: 1.4902
Epoch 8/10
686/686 [==============================] - 113s 165ms/step - loss: 1.5233 - val_loss: 1.4260
Epoch 9/10
686/686 [==============================] - 113s 165ms/step - loss: 1.4601 - val_loss: 1.3652
Epoch 10/10
686/686 [==============================] - 112s 163m

### 모델 평가

In [52]:
## 잘 만들어졌는지 평가하기 - 작문을 시켜보고 직접 평가
# generate_text함수 : 모델에게 시작 문장을 전달하면 모델이 시작 문장을 바탕으로 작문을 진행
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 일단 텐서로 변환합니다.
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 텍스트를 실제로 생성할때는 루프를 돌면서 단어 하나씩 생성해야 합니다.  (테스트 데이터셋 없음)
    while True:
        predict = model(test_tensor)  # 입력받은 문장의 텐서를 입력합니다. 
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1]   # 우리 모델이 예측한 마지막 단어가 바로 새롭게 생성한 단어가 됩니다. 

        # 우리 모델이 새롭게 예측한 단어를 입력 문장의 뒤에 붙여 줍니다. 
        test_tensor = tf.concat([test_tensor, 
                                 tf.expand_dims(predict_word, axis=0)], axis=-1)

        # 우리 모델이 <end>를 예측했거나, max_len에 도달하지 않았다면  while 루프를 또 돌면서 다음 단어를 예측해야 합니다.
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # 생성된 tensor 안에 있는 word index를 tokenizer.index_word 사전을 통해 실제 단어로 하나씩 변환합니다. 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated   # 이것이 최종적으로 모델이 생성한 자연어 문장입니다.

In [62]:
# 문장 생성 함수
generate_text(model, tokenizer, init_sentence="<start> I take", max_len=20)

'<start> i take it for a minute , <end> '

---
## 루브릭 평가 요구사항  
  
### 1) 가사 텍스트 생성 모델이 정상적으로 동작하는가? ?
- 가사 자동 생성 모델을 만들어 훈련시키고 테스트한 결과, 문장 생성 했을 때 그럴듯 한 문장으로 완성시켜 준다.

### 2)  데이터의 전처리와 데이터셋 구성 과정이 체계적으로 진행되었는가?  
- 정제 함수를 통해 특수문자를 제거하고, Tokenizer 패키지로 토큰화를 진행하였으며, 데이터 시퀸스 길이를 일정하게 맞추기 위한 padding 메소드를 활용하였다. 

### 3) 텍스트 생성모델의 validation loss가 2.2 이하로 낮아졌는가?  
- 텍스트 생성 모델의 validation loss 가 모두 2 이하였다.  
---

# 회고  
---- 
이전 프로젝트를 통해 자연어 처리에 익숙해져서 처음 보다는 조금 더 수월하게 프로젝트를 마무리 하였다. 마지막 테스트를 위한 문장 생성의 초기 단어를 계속 바꿔주며 결과값을 지켜보았다. 조금은 미숙한 문장이 나타날 때도 있었지만 그럴듯한 긴 문장도 만들어내니 신기하기도 하였다.   

약 한달정도의 프로젝트를 진행하니, 이제는 제출을 위한 학습을 하는 것이 아닌가 생각이 들었다. 내것으로 받아들이기 위해 코드 하나하나를 분석하고 학습하고 싶다.  
